## **Fast AI Experements Based on [this](https://towardsdatascience.com/transfer-learning-using-the-fastai-library-d686b238213e) blog**

trained models in this [google Drive Folder](https://drive.google.com/open?id=1bW0UjVudEarP5qTToxwDtcIKXX9iuM9L)

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# !unzip drive/My\ Drive/ammi-2020-convnets.zip 

# !mkdir models
# !cp -r drive/My\ Drive/data/models/* models/.


!cp -r models/* drive/My\ Drive/data/models/.


In [0]:
!pip install pretrainedmodels
!pip uninstall torch torchvision -y 
!pip install torch==1.4.0 torchvision==0.5.0 

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=288e624eac9b28a91b625a31b0ae9122802d2d076c207370c8e5d421852a169f
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels
Uninstalling torch-1.5.0+cu101:
  Successfully uninstalled torch-1.5.0+cu101
Uninstalling torchvision-0.6.0+cu101:
  Successfully uninstalled torchvision-0.6.0+cu101
     |████████████████████████████████| 753.4MB 22kB/s 
     |████████████████████████████████| 4.0MB 67kB/s 


# Importing Fast AI library

In [0]:
import os 
import pretrainedmodels

from tqdm import tqdm

In [0]:
from fastai import *
from fastai.vision import *

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import auc,roc_curve

from math import floor

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import os,glob
os.listdir('train/train/')

['cbb', 'healthy', 'cbsd', 'cmd', 'cgm']

In [0]:
healthy=glob.glob('train/train/healthy/*.*')
cmd=glob.glob('train/train/cmd/*.*')
cbsd=glob.glob('train/train/cbsd/*.*')
cgm=glob.glob('train/train/cgm/*.*')
cbb=glob.glob('train/train/cbb/*.*')

In [0]:
img_path=[]
img_path=img_path+healthy+cmd+cbsd+cgm+cbb
df=pd.DataFrame({'img_path':img_path})
df.shape

(5656, 1)

In [0]:
df.head()

,img_path
0,train/train/healthy/train-healthy-294.jpg
1,train/train/healthy/train-healthy-128.jpg
2,train/train/healthy/train-healthy-231.jpg
3,train/train/healthy/train-healthy-225.jpg
4,train/train/healthy/train-healthy-120.jpg


In [0]:
df['disease_class'] = df['img_path'].apply(lambda x: x.split("/")[3].split('.')[0].split('-')[1])
df['ext'] = df['img_path'].apply(lambda x: x.split(".")[-1])


In [0]:
df.tail(15)

,img_path,disease_class,ext
5641,train/train/cbb/train-cbb-303.jpg,cbb,jpg
5642,train/train/cbb/train-cbb-140.jpg,cbb,jpg
5643,train/train/cbb/train-cbb-111.jpg,cbb,jpg
5644,train/train/cbb/train-cbb-70.jpg,cbb,jpg
5645,train/train/cbb/train-cbb-327.jpg,cbb,jpg
5646,train/train/cbb/train-cbb-305.jpg,cbb,jpg
5647,train/train/cbb/train-cbb-313.jpg,cbb,jpg
5648,train/train/cbb/train-cbb-403.jpg,cbb,jpg
5649,train/train/cbb/train-cbb-294.jpg,cbb,jpg
5650,train/train/cbb/train-cbb-101.jpg,cbb,jpg


In [0]:
df.drop('ext',axis=1,inplace=True)
df.head()

,img_path,disease_class
0,train/train/healthy/train-healthy-294.jpg,healthy
1,train/train/healthy/train-healthy-128.jpg,healthy
2,train/train/healthy/train-healthy-231.jpg,healthy
3,train/train/healthy/train-healthy-225.jpg,healthy
4,train/train/healthy/train-healthy-120.jpg,healthy


In [0]:
df.shape

(5656, 2)

In [0]:
df.reset_index(inplace=True)

In [0]:
df.drop('index',axis=1,inplace=True)
df.head()

,img_path,disease_class
0,train/train/healthy/train-healthy-294.jpg,healthy
1,train/train/healthy/train-healthy-128.jpg,healthy
2,train/train/healthy/train-healthy-231.jpg,healthy
3,train/train/healthy/train-healthy-225.jpg,healthy
4,train/train/healthy/train-healthy-120.jpg,healthy


In [0]:
test_img_path=glob.glob('./test/test/0/*.*')
test_df=pd.DataFrame({'test_img_path':test_img_path})
test_df['ext'] = test_df['test_img_path'].apply(lambda x: x.split(".")[-1])
test_df.shape


(3774, 2)

In [0]:
test_df.drop('ext',axis=1,inplace=True)
test_df.reset_index(inplace=True)
test_df.drop('index',axis=1,inplace=True)
test_df.head()

,test_img_path
0,./test/test/0/test-img-2372.jpg
1,./test/test/0/test-img-2618.jpg
2,./test/test/0/test-img-2205.jpg
3,./test/test/0/test-img-3488.jpg
4,./test/test/0/test-img-3079.jpg


In [0]:
df['disease_class'].value_counts()

cmd        2658
cbsd       1443
cgm         773
cbb         466
healthy     316
Name: disease_class, dtype: int64

In [0]:
df.head()

,img_path,disease_class
0,train/train/healthy/train-healthy-294.jpg,healthy
1,train/train/healthy/train-healthy-128.jpg,healthy
2,train/train/healthy/train-healthy-231.jpg,healthy
3,train/train/healthy/train-healthy-225.jpg,healthy
4,train/train/healthy/train-healthy-120.jpg,healthy


In [0]:
def create_databunch(valid_idx):
    transform_kwargs = {"do_flip": True,
                    "flip_vert": True,
                    "max_rotate": 180,
                    "max_zoom": 1.1,
                    "max_lighting": 0.2,
                    "max_warp": 0.2,
                    "p_affine": 0.75,
                    "p_lighting": 0.7}
    test = ImageList.from_df(test_df,path='.')
    data = (ImageList.from_df(df, path='.')
                .split_by_idx(valid_idx)
                .label_from_df(cols='disease_class')
                .add_test(test)
                .transform(get_transforms(**transform_kwargs), size=256)
                .databunch(path='.', bs=32)
                .normalize(imagenet_stats)
                )
    return data

In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4,shuffle=True,random_state=123)
acc_val=[]
i = 0

final_preds1=np.zeros((df.shape[0],df.shape[1]))
for train_idx, valid_idx in kf.split(df):
      print("Validation set:--->  ",i+1)
      data = create_databunch(valid_idx)
      learn = cnn_learner(data,models.resnet152, metrics=[error_rate,accuracy])
      learn.load('resnet152_89_cross_all')
      learn.fit_one_cycle(9, slice(1e-4,1e-2))
      test_dl = learn.data.test_dl
      preds= accuracy(*learn.TTA())
      print('Prediction Completed in fold: {}  Accuracy:{}'.format(str(i),preds))
      final_preds1+=preds.numpy()

final_preds1 = final_preds1/4

print("The validation accuracy is: ",np.mean(final_preds1))

Validation set:--->   1


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.356512,0.195929,0.059406,0.940594,01:07
1,0.379130,0.276676,0.093352,0.906648,01:07
2,0.416373,0.423972,0.139321,0.860679,01:07
3,0.390237,0.317623,0.094059,0.905941,01:07
4,0.365929,0.371573,0.111740,0.888260,01:07
5,0.314019,0.313811,0.100424,0.899576,01:07
6,0.291696,0.301609,0.095474,0.904526,01:07
7,0.257521,0.286203,0.088402,0.911598,01:07
8,0.230446,0.286513,0.090523,0.909477,01:07


Prediction Completed in fold: 0  Accuracy:0.9193776249885559
Validation set:--->   1


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.326530,0.171650,0.055870,0.944130,01:06
1,0.375008,0.230022,0.076379,0.923621,01:07
2,0.423207,0.315692,0.101132,0.898868,01:06
3,0.401047,0.366853,0.127298,0.872702,01:06
4,0.336933,0.307049,0.095474,0.904526,01:06
5,0.323307,0.301422,0.101132,0.898868,01:06
6,0.271701,0.300438,0.091231,0.908769,01:06
7,0.257074,0.278040,0.088402,0.911598,01:06
8,0.239602,0.276561,0.086280,0.913720,01:07


Prediction Completed in fold: 0  Accuracy:0.9179632067680359
Validation set:--->   1


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.317859,0.199990,0.074965,0.925035,01:07
1,0.365849,0.288577,0.094767,0.905233,01:06
2,0.408640,0.333806,0.112447,0.887553,01:06
3,0.395923,0.333274,0.104668,0.895332,01:07
4,0.394630,0.319508,0.103253,0.896747,01:06
5,0.313580,0.322122,0.103253,0.896747,01:07
6,0.285999,0.320107,0.100424,0.899576,01:07
7,0.270729,0.289058,0.091938,0.908062,01:07
8,0.236475,0.287662,0.094059,0.905941,01:06


Prediction Completed in fold: 0  Accuracy:0.909476637840271
Validation set:--->   1


epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.244626,0.371011,0.110325,0.889675,01:07
1,0.325691,0.522259,0.154173,0.845827,01:07
2,0.366985,0.517574,0.146393,0.853607,01:07
3,0.398165,0.460404,0.136492,0.863508,01:07
4,0.327180,0.453533,0.137907,0.862093,01:07
5,0.307096,0.412243,0.126591,0.873409,01:07
6,0.269445,0.492939,0.120934,0.879066,01:07
7,0.245376,0.411478,0.120226,0.879774,01:06
8,0.207456,0.422047,0.121641,0.878359,01:07


Prediction Completed in fold: 0  Accuracy:0.8875530362129211
The validation accuracy is:  0.908592626452446


In [0]:
learn.save("resnet152_89_cross_all_90", return_path=True)

PosixPath('models/resnet152_89_cross_all_90.pth')

In [0]:
#Prediction of the Test Set
preds_test, y_test = learn.get_preds(DatasetType.Test)

In [0]:
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir('test/test/0/') ]})
for i, c in enumerate(learn.data.classes):
  submission[c] = preds_test[:,i]
submission = submission[['Category']] # Get same order as sample sub
submission.head()

In [0]:
submission.to_csv('/content/drive/My Drive/ICLR/submission.csv', index = False)